# **NYU Wagner - Python Coding for Public Policy**
# Class 5: APIs

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every pull; higher barrier to entry (reading documentation); subject to availability and performance of API

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

Pull table from [Wikipedia's list of countries and dependencies by population](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population):

In [1]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population', match='Rank')
pop = tables[0]
pop

,Rank,Country / Dependency,Region,Population,Percentage of the world,Date,Source (official or from the United Nations),Notes
0,–,World,NaN,7950095000,100%,14 May 2022,UN projection[1],NaN
1,1,China,Asia,1412600000,NaN,31 Dec 2021,National annual estimate[2],The population figure refers to mainland China...
2,2,India,Asia,1376278850,NaN,14 May 2022,National population clock[3],The figure includes the population of Indian-a...
3,3,United States,Americas,332678288,NaN,14 May 2022,National population clock[4],The figure includes the 50 states and the Dist...
4,4,Indonesia,Asia[b],272248500,NaN,1 Jul 2021,National annual estimate[5],NaN
...,...,...,...,...,...,...,...,...
237,–,Niue (New Zealand),Oceania,1549,NaN,1 Jul 2021,National annual projection[93],NaN
238,–,Tokelau (New Zealand),Oceania,1501,NaN,1 Jul 2021,National annual projection[93],NaN
239,195,Vatican City,Europe,825,NaN,1 Feb 2019,Monthly national estimate[197],The total population of 825 consisted of 453 r...
240,–,Cocos (Keeling) Islands (Australia),Oceania,573,NaN,30 Jun 2020,National annual estimate[196],NaN


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Anchors aren't relevant

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

In [2]:
import requests

params = {
    'api_key': 'DEMO_KEY',
    'q': 'Jimmy McMillan',
    'sort': '-first_file_date'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()
data

{'api_version': '1.0',
 'results': [{'incumbent_challenge_full': 'Open seat',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'load_date': '2018-02-17T09:16:20+00:00',
   'candidate_status': 'N',
   'party_full': 'REPUBLICAN PARTY',
   'last_f2_date': '2015-10-13',
   'last_file_date': '2015-10-13',
   'election_years': [2016],
   'inactive_election_years': None,
   'district_number': 0,
   'district': '00',
   'party': 'REP',
   'incumbent_challenge': 'O',
   'office_full': 'President',
   'candidate_id': 'P60016805',
   'flags': 'P60016805',
   'candidate_inactive': False,
   'election_districts': ['00'],
   'cycles': [2016, 2018],
   'federal_funds_flag': False,
   'office': 'P',
   'first_file_date': '2015-10-13',
   'has_raised_funds': False,
   'state': 'US',
   'active_through': 2016},
  {'incumbent_challenge_full': 'Challenger',
   'name': 'MCMILLAN, JIMMY (AKA) JAMES ',
   'load_date': '2021-12-08T06:50:50+00:00',
   'candidate_status': 'N',
   'party_full': 'REPUBLICA

### Retrieving nested data

In [3]:
data['results'][0]['name']

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### [Homework 5](https://padmgp-4506-spring.rcnyu.org/user-redirect/notebooks/class_materials/hw_5.ipynb)

As an in-class exercise, start with geocoding your own address.

### Reading into a DataFrame

In [4]:
params = {
    'api_key': 'DEMO_KEY',
    'has_raised_funds': 'true'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()

pd.DataFrame(data['results'])

,office_full,active_through,last_file_date,flags,district_number,name,state,election_years,incumbent_challenge,federal_funds_flag,...,cycles,incumbent_challenge_full,load_date,office,first_file_date,candidate_status,inactive_election_years,party,candidate_id,party_full
0,House,2022,2021-12-27,H2CO07170,7,"AADLAND, ERIK",CO,[2022],O,False,...,[2022],Open seat,2022-01-13T01:40:22+00:00,H,2021-12-27,N,None,REP,H2CO07170,REPUBLICAN PARTY
1,House,2022,2022-03-21,H2UT03280,3,"AALDERS, TIM",UT,[2022],C,False,...,[2022],Challenger,2022-04-13T21:10:09+00:00,H,2020-03-24,C,None,REP,H2UT03280,REPUBLICAN PARTY
2,Senate,2018,2018-04-23,S2UT00229,0,"AALDERS, TIMOTHY NOEL",UT,"[2012, 2018]",O,False,...,"[2012, 2014, 2016, 2018, 2020]",Open seat,2019-03-27T16:02:41+00:00,S,2012-02-08,P,None,CON,S2UT00229,CONSTITUTION PARTY
3,House,2020,2019-10-17,H0TX22260,22,"AALOORI, BANGAR REDDY",TX,[2020],O,False,...,[2020],Open seat,2020-03-18T21:13:37+00:00,H,2019-10-17,C,None,REP,H0TX22260,REPUBLICAN PARTY
4,House,1978,1978-07-05,H6PA16106,16,"AAMODT, NORMAN O.",PA,"[1976, 1978]",None,False,...,"[1976, 1978, 1980]",None,2002-03-30T00:00:00+00:00,H,1976-04-12,P,None,REP,H6PA16106,REPUBLICAN PARTY
5,House,2012,2012-02-22,H2CA01110,1,"AANESTAD, SAMUEL",CA,[2012],C,False,...,"[2012, 2014, 2016]",Challenger,2013-04-26T09:04:30+00:00,H,2012-02-22,P,None,REP,H2CA01110,REPUBLICAN PARTY
6,House,2018,2017-04-26,H8CO06237,6,"AARESTAD, DAVID",CO,[2018],C,False,...,[2018],Challenger,2017-08-01T20:57:28+00:00,H,2017-04-26,C,None,DEM,H8CO06237,DEMOCRATIC PARTY
7,President,2008,2007-03-13,P80002926,0,"AARON, LAURA DAVIS",US,[2008],O,False,...,"[2006, 2008, 2010, 2012, 2014, 2016]",Open seat,2016-11-17T06:10:48+00:00,P,2005-10-12,N,None,DEM,P80002926,DEMOCRATIC PARTY
8,House,2022,2022-01-24,H2CA30291,32,"AAZAMI, SHERVIN",CA,[2022],C,False,...,[2022],Challenger,2022-01-25T20:59:17+00:00,H,2021-01-16,C,None,DEM,H2CA30291,DEMOCRATIC PARTY
9,House,2000,2000-02-02,H0MA01024,1,"ABAIR, PETER JON",MA,[2000],C,False,...,"[2000, 2002, 2004]",Challenger,2002-04-12T00:00:00+00:00,H,2000-02-02,P,None,REP,H0MA01024,REPUBLICAN PARTY


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2022, 5, 15, 2, 50, 31, 125499)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2022, 5, 8, 2, 50, 31, 125499)

In [7]:
start.isoformat()

'2022-05-08T02:50:31.125499'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = 'erm2-nwe9'
params = {
    '$where': f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"
}

url = f'https://data.cityofnewyork.us/resource/{data_id}.json'
response = requests.get(url, params=params)
data = response.json()

data

[]

In [9]:
pd.DataFrame(data)

""


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## Homework 6

In real/ideal world, start with specific question and find data to answer it:

![project flow](extras/img/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](extras/img/projectflow_amended.png)

[Homework 6](https://padmgp-4506-spring.rcnyu.org/user-redirect/notebooks/class_materials/hw_6.ipynb)

## No homework/resubmissions will be accepted after Sunday 5/15 at 6:45pm ET

In other words, homework 6 cannot be late.

## Lecture 6